In [1]:
import sys
import numpy as np
import keras #from tensorflow.python import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras import optimizers
from keras import regularizers
from keras import backend as K
from sklearn.model_selection import StratifiedKFold

import seaborn.apionly as sn
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support

%matplotlib inline

print(keras.__version__)

Using TensorFlow backend.


Instructions for updating:
Use the retry module or similar alternatives.
2.0.8


C:\Users\Brayan\Anaconda3\envs\tensorflow-gpu\lib\site-packages\seaborn\apionly.py:6: UserWarning: As seaborn no longer sets a default style on import, the seaborn.apionly module is deprecated. It will be removed in a future version.
  warnings.warn(msg, UserWarning)


In [2]:
#### 1 -> funciona bien con 10cv -> 94.1
# 2 -> no hay diferencias con 1 -> 93.9
#### 3 -> funciona mejor con 5cv -> 94.x, 10cv-> 93.8
#### 4 -> funciona fatal -> 91
# 5 -> casi igual que 1 -> 93.6
# 6 -> F1 score -> 93.20

NORM = 'stdnorm'
DISTRIBUTION = 7
FILLING = 'cero2mean'

LABELS_OUT_FILE = './datasets/labels-whole-t' + str(DISTRIBUTION) + '-' + FILLING + '-' + NORM + '.npy'
IMAGES_OUT_FILE = './datasets/images-whole-t' + str(DISTRIBUTION) + '-' + FILLING + '-' + NORM + '.npy'

FINGERS = 3
TACTILE_IMAGE_ROWS = -1
TACTILE_IMAGE_COLS = -1

if DISTRIBUTION == 1:
    TACTILE_IMAGE_ROWS = 8
    TACTILE_IMAGE_COLS = 9
elif DISTRIBUTION == 2:
    TACTILE_IMAGE_ROWS = 8
    TACTILE_IMAGE_COLS = 7
elif DISTRIBUTION == 3:
    TACTILE_IMAGE_ROWS = 6
    TACTILE_IMAGE_COLS = 7
elif DISTRIBUTION == 4:
    TACTILE_IMAGE_ROWS = 4
    TACTILE_IMAGE_COLS = 7
elif DISTRIBUTION == 5:
    TACTILE_IMAGE_ROWS = 6
    TACTILE_IMAGE_COLS = 5
elif DISTRIBUTION == 6:
    TACTILE_IMAGE_ROWS = 6
    TACTILE_IMAGE_COLS = 5
elif DISTRIBUTION == 7:
    TACTILE_IMAGE_ROWS = 12
    TACTILE_IMAGE_COLS = 11

In [3]:
labels = np.load(LABELS_OUT_FILE)
labels_cat = np_utils.to_categorical(labels, num_classes=2)
tactile_images = np.load(IMAGES_OUT_FILE)

In [4]:
print(tactile_images[0])

[[[ 0.20416431  0.20416431  0.48778092  0.77139752  0.79018008
    0.80896263  0.73852805  0.66809346  0.27647716 -0.11513915
   -0.11513915]
  [ 0.20416431  0.48778092  0.77139752  0.77139752  0.80896263
    0.80896263  0.80896263  0.66809346  0.66809346  0.27647716
   -0.11513915]
  [ 1.25260662  0.77139752  0.77139752  0.77515403  0.79393659
    0.80896263  0.72256287  0.65212829  0.66809346  0.66809346
    0.12133324]
  [ 2.39984517  2.39984517  1.11429657  0.77891054  0.77891054
    0.74134543  0.63616311  0.63616311  0.39750725 -0.11138264
   -0.11138264]
  [ 2.39984517  1.67953413  0.95922309  0.77891054  0.58075457
    0.59922409  0.50938086  0.63616311  0.05766037 -0.02686113
   -0.11138264]
  [ 1.67953413  0.95922309  0.62301532  0.28680756  0.33470308
    0.3825986  -0.28793867 -0.95847593 -0.45040778  0.05766037
   -0.02686113]
  [ 0.43143325  0.43143325  0.62301532  0.28680756  0.33470308
    0.3825986  -0.28793867 -0.95847593 -0.45040778 -0.08226967
   -0.08226967]
  [-0.

# Single test

In [5]:
folds = 10
kfold = StratifiedKFold(n_splits=folds, shuffle=True)
cv_accuracies = []
cv_precision = []
cv_recall = []
cv_f1_score = []

for index, (train_indices, val_indices) in enumerate(kfold.split(tactile_images, labels)):
    print("Training fold " + str(index + 1) + "/" + str(folds))

    # split data
    images_train, images_val = tactile_images[train_indices], tactile_images[val_indices]
    labels_train, labels_val = labels_cat[train_indices], labels_cat[val_indices]  
    
    # build model
    epochs = 500
    batch = 32
    
    learning_rate = 0.0001
    epsilon = 1e-08
    decay_rate = 0.003
    
    l2_reg = 0.01
    drop_prob = 0.2
        
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=(FINGERS, TACTILE_IMAGE_ROWS, TACTILE_IMAGE_COLS), 
                     data_format='channels_first',
                     use_bias=False, kernel_regularizer=regularizers.l2(l2_reg)))
    model.add(BatchNormalization(axis=1))
    model.add(Activation('relu'))
    
    model.add(Conv2D(64, (3, 3), data_format='channels_first',
                     use_bias=False, kernel_regularizer=regularizers.l2(l2_reg)))
    model.add(BatchNormalization(axis=1))
    model.add(Activation('relu'))
    
    model.add(Flatten())
    
    model.add(Dense(1024, kernel_regularizer=regularizers.l2(l2_reg)))
    model.add(BatchNormalization(axis=1))
    model.add(Activation('relu'))
    model.add(Dropout(drop_prob))
    
    model.add(Dense(2, activation='softmax'))

    # compile and train
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.Adam(lr=learning_rate, epsilon=epsilon, decay=decay_rate), 
                  metrics=['accuracy'])
    model.fit(images_train, labels_train, epochs=epochs, batch_size=batch, verbose=1)
    
    # evaluate
    print("# # # Evaluating cv-fold...")
    scores = model.evaluate(images_val, labels_val, verbose=1)
    cv_accuracies.append(scores[1] * 100)
    
    predictions = model.predict(images_val)
    [precision, recall, f1_score, _] = precision_recall_fscore_support(np.argmax(labels_val, axis=1), 
                                                                          np.argmax(predictions, axis=1), 
                                                                          average='binary', pos_label=1)
    
    cv_precision.append(precision * 100)
    cv_recall.append(recall * 100)
    cv_f1_score.append(f1_score * 100)
    
    print("\naccuracy:", scores[1] * 100)
    print("precision:", precision * 100)
    print("recall:", recall * 100)
    print("f1_score:", f1_score * 100)
    
print("\nFinal cross-validation score:")
print(np.mean(cv_accuracies), "+/-", np.std(cv_accuracies))
print(np.mean(cv_precision), "+/-", np.std(cv_precision))
print(np.mean(cv_recall), "+/-", np.std(cv_recall))
print(np.mean(cv_f1_score), "+/-", np.std(cv_f1_score))

#93.0960318615 +/- 1.44583890067
#92.1509533078 +/- 1.58488408132
#94.2944143701 +/- 2.09475574588
#93.1961448124 +/- 1.44722062549

Training fold 1/10
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Epoch 1/500
2294/2294 [==============================] - 5s - loss: 14.7590 - acc: 0.8422     
Epoch 2/500
2294/2294 [==============================] - 0s - loss: 11.4768 - acc: 0.8980     
Epoch 3/500
2294/2294 [==============================] - 0s - loss: 9.3973 - acc: 0.9172     
Epoch 4/500
2294/2294 [==============================] - 0s - loss: 8.0031 - acc: 0.9119     
Epoch 5/500
2294/2294 [==============================] - 0s - loss: 7.0066 - acc: 0.9285     
Epoch 6/500
2294/2294 [==============================] - 0s - loss: 6.2504 - acc: 0.9398     
Epoch 7/500
2294/2294 [==============================] - 0s - loss: 5.6785 - acc: 0.9372     
Epoch 8/500
2294/2294 [==============================] - 0s - loss: 5.2275 - acc: 0.9433     
Epoch 9/

2294/2294 [==============================] - 0s - loss: 1.1344 - acc: 0.9974     
Epoch 160/500
2294/2294 [==============================] - 0s - loss: 1.1311 - acc: 0.9969     
Epoch 161/500
2294/2294 [==============================] - 0s - loss: 1.1291 - acc: 0.9983     
Epoch 162/500
2294/2294 [==============================] - 0s - loss: 1.1260 - acc: 0.9969     
Epoch 163/500
2294/2294 [==============================] - 0s - loss: 1.1192 - acc: 0.9987     
Epoch 164/500
2294/2294 [==============================] - 0s - loss: 1.1176 - acc: 0.9974     
Epoch 165/500
2294/2294 [==============================] - 0s - loss: 1.1130 - acc: 0.9983     
Epoch 166/500
2294/2294 [==============================] - 0s - loss: 1.1114 - acc: 0.9974     
Epoch 167/500
2294/2294 [==============================] - 0s - loss: 1.1093 - acc: 0.9978     
Epoch 168/500
2294/2294 [==============================] - 0s - loss: 1.1079 - acc: 0.9978     
Epoch 169/500
2294/2294 [=============================

2294/2294 [==============================] - 0s - loss: 0.8209 - acc: 0.9987     
Epoch 328/500
2294/2294 [==============================] - 0s - loss: 0.8185 - acc: 0.9991     
Epoch 329/500
2294/2294 [==============================] - 0s - loss: 0.8187 - acc: 0.9996     
Epoch 330/500
2294/2294 [==============================] - 0s - loss: 0.8185 - acc: 0.9991     
Epoch 331/500
2294/2294 [==============================] - 0s - loss: 0.8174 - acc: 0.9983     
Epoch 332/500
2294/2294 [==============================] - 0s - loss: 0.8145 - acc: 0.9991     
Epoch 333/500
2294/2294 [==============================] - 0s - loss: 0.8139 - acc: 0.9991     
Epoch 334/500
2294/2294 [==============================] - 0s - loss: 0.8127 - acc: 0.9996     
Epoch 335/500
2294/2294 [==============================] - 0s - loss: 0.8129 - acc: 0.9987     
Epoch 336/500
2294/2294 [==============================] - 0s - loss: 0.8135 - acc: 0.9983     
Epoch 337/500
2294/2294 [=============================

2294/2294 [==============================] - 0s - loss: 0.7395 - acc: 0.9991     
Epoch 413/500
2294/2294 [==============================] - 0s - loss: 0.7391 - acc: 0.9991     
Epoch 414/500
2294/2294 [==============================] - 0s - loss: 0.7404 - acc: 0.9991     
Epoch 415/500
2294/2294 [==============================] - 0s - loss: 0.7368 - acc: 0.9996     
Epoch 416/500
2294/2294 [==============================] - 1s - loss: 0.7374 - acc: 0.9987     
Epoch 417/500
2294/2294 [==============================] - 0s - loss: 0.7366 - acc: 0.9991     
Epoch 418/500
2294/2294 [==============================] - 0s - loss: 0.7356 - acc: 0.9991     
Epoch 419/500
2294/2294 [==============================] - 0s - loss: 0.7349 - acc: 0.9996     
Epoch 420/500
2294/2294 [==============================] - 0s - loss: 0.7339 - acc: 0.9987     
Epoch 421/500
2294/2294 [==============================] - 0s - loss: 0.7337 - acc: 0.9991     
Epoch 422/500
2294/2294 [=============================

2294/2294 [==============================] - 0s - loss: 1.5948 - acc: 0.9948     
Epoch 82/500
2294/2294 [==============================] - 0s - loss: 1.5846 - acc: 0.9939     
Epoch 83/500
2294/2294 [==============================] - 0s - loss: 1.5770 - acc: 0.9930     
Epoch 84/500
2294/2294 [==============================] - 0s - loss: 1.5712 - acc: 0.9956     
Epoch 85/500
2294/2294 [==============================] - 0s - loss: 1.5588 - acc: 0.9943     
Epoch 86/500
2294/2294 [==============================] - 0s - loss: 1.5551 - acc: 0.9935     
Epoch 87/500
2294/2294 [==============================] - 0s - loss: 1.5442 - acc: 0.9948     
Epoch 88/500
2294/2294 [==============================] - 0s - loss: 1.5397 - acc: 0.9935     
Epoch 89/500
2294/2294 [==============================] - 0s - loss: 1.5276 - acc: 0.9952     
Epoch 90/500
2294/2294 [==============================] - 0s - loss: 1.5240 - acc: 0.9943     
Epoch 91/500
2294/2294 [==============================] - 0s - 

...

2294/2294 [==============================] - 0s - loss: 0.6997 - acc: 0.9991     
Epoch 452/500
2294/2294 [==============================] - 0s - loss: 0.6992 - acc: 1.0000     
Epoch 453/500
2294/2294 [==============================] - 0s - loss: 0.6990 - acc: 0.9987     
Epoch 454/500
2294/2294 [==============================] - 0s - loss: 0.6989 - acc: 0.9991     
Epoch 455/500
2294/2294 [==============================] - 0s - loss: 0.6966 - acc: 0.9996     
Epoch 456/500
2294/2294 [==============================] - 0s - loss: 0.6953 - acc: 0.9996     
Epoch 457/500
2294/2294 [==============================] - 0s - loss: 0.6950 - acc: 1.0000     
Epoch 458/500
2294/2294 [==============================] - 0s - loss: 0.6937 - acc: 0.9991     
Epoch 459/500
2294/2294 [==============================] - 0s - loss: 0.6951 - acc: 0.9991     
Epoch 460/500
2294/2294 [==============================] - 0s - loss: 0.6920 - acc: 1.0000     
Epoch 461/500
2294/2294 [=============================

2295/2295 [==============================] - 0s - loss: 1.3084 - acc: 0.9952     
Epoch 119/500
2295/2295 [==============================] - 0s - loss: 1.3018 - acc: 0.9969     
Epoch 120/500
2295/2295 [==============================] - 0s - loss: 1.2938 - acc: 0.9983     
Epoch 121/500
2295/2295 [==============================] - 0s - loss: 1.2941 - acc: 0.9965     
Epoch 122/500
2295/2295 [==============================] - 0s - loss: 1.2861 - acc: 0.9974     
Epoch 123/500
2295/2295 [==============================] - 0s - loss: 1.2811 - acc: 0.9965     
Epoch 124/500
2295/2295 [==============================] - 0s - loss: 1.2810 - acc: 0.9952     
Epoch 125/500
2295/2295 [==============================] - 0s - loss: 1.2734 - acc: 0.9952     
Epoch 126/500
2295/2295 [==============================] - 0s - loss: 1.2702 - acc: 0.9961     
Epoch 127/500
2295/2295 [==============================] - 0s - loss: 1.2633 - acc: 0.9983     
Epoch 128/500
2295/2295 [=============================

2295/2295 [==============================] - 0s - loss: 0.8611 - acc: 0.9991     
Epoch 288/500
2295/2295 [==============================] - 0s - loss: 0.8604 - acc: 0.9983     
Epoch 289/500
2295/2295 [==============================] - 0s - loss: 0.8588 - acc: 0.9991     
Epoch 290/500
2295/2295 [==============================] - 0s - loss: 0.8579 - acc: 0.9983     
Epoch 291/500
2295/2295 [==============================] - 0s - loss: 0.8558 - acc: 0.9991     
Epoch 292/500
2295/2295 [==============================] - 0s - loss: 0.8538 - acc: 0.9991     - ETA: 0s - loss: 0.8541 - acc: 0.99
Epoch 293/500
2295/2295 [==============================] - 0s - loss: 0.8533 - acc: 0.9983     
Epoch 294/500
2295/2295 [==============================] - 0s - loss: 0.8514 - acc: 0.9996     
Epoch 295/500
2295/2295 [==============================] - 0s - loss: 0.8493 - acc: 0.9996     
Epoch 296/500
2295/2295 [==============================] - 0s - loss: 0.8466 - acc: 1.0000     
Epoch 297/500
2295

2295/2295 [==============================] - 0s - loss: 0.7613 - acc: 1.0000     
Epoch 372/500
2295/2295 [==============================] - 0s - loss: 0.7609 - acc: 0.9996     
Epoch 373/500
2295/2295 [==============================] - 0s - loss: 0.7587 - acc: 1.0000     
Epoch 374/500
2295/2295 [==============================] - 0s - loss: 0.7582 - acc: 1.0000     
Epoch 375/500
2295/2295 [==============================] - 0s - loss: 0.7571 - acc: 1.0000     
Epoch 376/500
2295/2295 [==============================] - 0s - loss: 0.7573 - acc: 0.9996     
Epoch 377/500
2295/2295 [==============================] - 0s - loss: 0.7548 - acc: 0.9996     
Epoch 378/500
2295/2295 [==============================] - 0s - loss: 0.7527 - acc: 0.9996     
Epoch 379/500
2295/2295 [==============================] - 0s - loss: 0.7556 - acc: 0.9991     
Epoch 380/500
2295/2295 [==============================] - 0s - loss: 0.7528 - acc: 0.9996     
Epoch 381/500
2295/2295 [=============================

2295/2295 [==============================] - 0s - loss: 0.6870 - acc: 0.9996     
Epoch 457/500
2295/2295 [==============================] - 0s - loss: 0.6874 - acc: 0.9987     
Epoch 458/500
2295/2295 [==============================] - 0s - loss: 0.6874 - acc: 1.0000     
Epoch 459/500
2295/2295 [==============================] - 0s - loss: 0.6857 - acc: 0.9996     
Epoch 460/500
2295/2295 [==============================] - 0s - loss: 0.6849 - acc: 0.9996     
Epoch 461/500
2295/2295 [==============================] - 0s - loss: 0.6853 - acc: 0.9991     
Epoch 462/500
2295/2295 [==============================] - 0s - loss: 0.6833 - acc: 0.9996     
Epoch 463/500
2295/2295 [==============================] - 0s - loss: 0.6821 - acc: 1.0000     
Epoch 464/500
2295/2295 [==============================] - 0s - loss: 0.6833 - acc: 0.9991     
Epoch 465/500
2295/2295 [==============================] - 0s - loss: 0.6811 - acc: 0.9996     
Epoch 466/500
2295/2295 [=============================

In [7]:
# BEST

folds = 10
kfold = StratifiedKFold(n_splits=folds, shuffle=True)
cv_accuracies = []
cv_precision = []
cv_recall = []
cv_f1_score = []

for index, (train_indices, val_indices) in enumerate(kfold.split(tactile_images, labels)):
    print("Training fold " + str(index + 1) + "/" + str(folds))

    # split data
    images_train, images_val = tactile_images[train_indices], tactile_images[val_indices]
    labels_train, labels_val = labels_cat[train_indices], labels_cat[val_indices]  
    
    # build model
    epochs = 200
    batch = 32
    
    learning_rate = 0.0001
    epsilon = 1e-08
    decay_rate = 0.005
    
    l2_reg = 0.05
    drop_prob = 0.0
    
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=(FINGERS, TACTILE_IMAGE_ROWS, TACTILE_IMAGE_COLS), 
                     data_format='channels_first',
                     use_bias=False, kernel_regularizer=regularizers.l2(l2_reg)))
    model.add(BatchNormalization(axis=1))
    model.add(Activation('relu'))
        
    model.add(Flatten())
    
    model.add(Dense(1024, kernel_regularizer=regularizers.l2(l2_reg)))
    model.add(BatchNormalization(axis=1))
    model.add(Activation('relu'))
    model.add(Dropout(drop_prob))
    
    model.add(Dense(2, activation='softmax'))

    # compile and train
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.Adam(lr=learning_rate, epsilon=epsilon, decay=decay_rate), 
                  metrics=['accuracy'])
    model.fit(images_train, labels_train, epochs=epochs, batch_size=batch, verbose=1)
    
    # evaluate
    print("# # # Evaluating cv-fold...")
    scores = model.evaluate(images_val, labels_val, verbose=1)
    cv_accuracies.append(scores[1] * 100)
    
    predictions = model.predict(images_val)
    [precision, recall, f1_score, _] = precision_recall_fscore_support(np.argmax(labels_val, axis=1), 
                                                                          np.argmax(predictions, axis=1), 
                                                                          average='binary', pos_label=1)
    
    cv_precision.append(precision * 100)
    cv_recall.append(recall * 100)
    cv_f1_score.append(f1_score * 100)
    
    print("\naccuracy:", scores[1] * 100)
    print("precision:", precision * 100)
    print("recall:", recall * 100)
    print("f1_score:", f1_score * 100)
    
print("\nFinal cross-validation score:")
print(np.mean(cv_accuracies), "+/-", np.std(cv_accuracies))
print(np.mean(cv_precision), "+/-", np.std(cv_precision))
print(np.mean(cv_recall), "+/-", np.std(cv_recall))
print(np.mean(cv_f1_score), "+/-", np.std(cv_f1_score))

#93.487416825 +/- 1.01206064896
#92.0686512508 +/- 2.13599373378
#95.3088090551 +/- 1.74696319943
#93.629627153 +/- 0.949247598202

Training fold 1/10
Epoch 1/200
2294/2294 [==============================] - 1s - loss: 62.5582 - acc: 0.8527     
Epoch 2/200
2294/2294 [==============================] - 0s - loss: 43.3128 - acc: 0.9032     
Epoch 3/200
2294/2294 [==============================] - 0s - loss: 32.2776 - acc: 0.9146     
Epoch 4/200
2294/2294 [==============================] - 0s - loss: 25.2375 - acc: 0.9276     
Epoch 5/200
2294/2294 [==============================] - 0s - loss: 20.4431 - acc: 0.9228     
Epoch 6/200
2294/2294 [==============================] - 0s - loss: 17.0020 - acc: 0.9276     
Epoch 7/200
2294/2294 [==============================] - 0s - loss: 14.4201 - acc: 0.9429     
Epoch 8/200
2294/2294 [==============================] - 0s - loss: 12.4501 - acc: 0.9451     
Epoch 9/200
2294/2294 [==============================] - 0s - loss: 10.8955 - acc: 0.9455     
Epoch 10/200
2294/2294 [==============================] - 0s - loss: 9.6541 - acc: 0.9468     
Epoch 11/200
2294/2294 [=======

2294/2294 [==============================] - 0s - loss: 1.3744 - acc: 0.9891     
Epoch 88/200
2294/2294 [==============================] - 0s - loss: 1.3613 - acc: 0.9908     
Epoch 89/200
2294/2294 [==============================] - 0s - loss: 1.3585 - acc: 0.9878     
Epoch 90/200
2294/2294 [==============================] - 0s - loss: 1.3469 - acc: 0.9900     
Epoch 91/200
2294/2294 [==============================] - 0s - loss: 1.3425 - acc: 0.9908     
Epoch 92/200
2294/2294 [==============================] - 0s - loss: 1.3311 - acc: 0.9904     
Epoch 93/200
2294/2294 [==============================] - 0s - loss: 1.3232 - acc: 0.9930     
Epoch 94/200
2294/2294 [==============================] - 0s - loss: 1.3227 - acc: 0.9847     
Epoch 95/200
2294/2294 [==============================] - 0s - loss: 1.3114 - acc: 0.9882     
Epoch 96/200
2294/2294 [==============================] - 0s - loss: 1.3039 - acc: 0.9874     
Epoch 97/200
2294/2294 [==============================] - 0s - 

2294/2294 [==============================] - 0s - loss: 1.8078 - acc: 0.9808     
Epoch 57/200
2294/2294 [==============================] - 0s - loss: 1.7859 - acc: 0.9826     
Epoch 58/200
2294/2294 [==============================] - 0s - loss: 1.7688 - acc: 0.9813     
Epoch 59/200
2294/2294 [==============================] - 0s - loss: 1.7476 - acc: 0.9834     
Epoch 60/200
2294/2294 [==============================] - 0s - loss: 1.7241 - acc: 0.9856     
Epoch 61/200
2294/2294 [==============================] - 0s - loss: 1.7112 - acc: 0.9826     
Epoch 62/200
2294/2294 [==============================] - 0s - loss: 1.6957 - acc: 0.9834     
Epoch 63/200
2294/2294 [==============================] - 0s - loss: 1.6787 - acc: 0.9821     
Epoch 64/200
2294/2294 [==============================] - 0s - loss: 1.6582 - acc: 0.9882     
Epoch 65/200
2294/2294 [==============================] - 0s - loss: 1.6462 - acc: 0.9813     
Epoch 66/200
2294/2294 [==============================] - 0s - 

2294/2294 [==============================] - 0s - loss: 3.5892 - acc: 0.9677     
Epoch 25/200
2294/2294 [==============================] - 0s - loss: 3.4345 - acc: 0.9738     
Epoch 26/200
2294/2294 [==============================] - 0s - loss: 3.3155 - acc: 0.9660     
Epoch 27/200
2294/2294 [==============================] - 0s - loss: 3.1786 - acc: 0.9756     
Epoch 28/200
2294/2294 [==============================] - 0s - loss: 3.0792 - acc: 0.9721     
Epoch 29/200
2294/2294 [==============================] - 0s - loss: 2.9729 - acc: 0.9747     
Epoch 30/200
2294/2294 [==============================] - 0s - loss: 2.8826 - acc: 0.9747     
Epoch 31/200
2294/2294 [==============================] - 0s - loss: 2.8011 - acc: 0.9695     
Epoch 32/200
2294/2294 [==============================] - 0s - loss: 2.7200 - acc: 0.9765     
Epoch 33/200
2294/2294 [==============================] - 0s - loss: 2.6379 - acc: 0.9799     
Epoch 34/200
2294/2294 [==============================] - 0s - 

2294/2294 [==============================] - 0s - loss: 0.8975 - acc: 0.9926     
Epoch 196/200
2294/2294 [==============================] - 0s - loss: 0.8957 - acc: 0.9908     
Epoch 197/200
2294/2294 [==============================] - 0s - loss: 0.8922 - acc: 0.9952     
Epoch 198/200
2294/2294 [==============================] - 0s - loss: 0.8893 - acc: 0.9926     
Epoch 199/200
2294/2294 [==============================] - 0s - loss: 0.8879 - acc: 0.9965     
Epoch 200/200
2294/2294 [==============================] - 0s - loss: 0.8852 - acc: 0.9943     
# # # Evaluating cv-fold...
 32/255 [==>...........................] - ETA: 2s
accuracy: 91.76470555511176
precision: 88.48920863309353
recall: 96.09375
f1_score: 92.13483146067416
Training fold 4/10
Epoch 1/200
2294/2294 [==============================] - 1s - loss: 62.5696 - acc: 0.8627     
Epoch 2/200
2294/2294 [==============================] - 0s - loss: 43.2262 - acc: 0.8997     
Epoch 3/200
2294/2294 [=========================

...

2294/2294 [==============================] - 0s - loss: 0.9468 - acc: 0.9930     
Epoch 186/200
2294/2294 [==============================] - 0s - loss: 0.9464 - acc: 0.9943     
Epoch 187/200
2294/2294 [==============================] - 0s - loss: 0.9456 - acc: 0.9917     
Epoch 188/200
2294/2294 [==============================] - 0s - loss: 0.9401 - acc: 0.9926     
Epoch 189/200
2294/2294 [==============================] - 0s - loss: 0.9389 - acc: 0.9922     
Epoch 190/200
2294/2294 [==============================] - 0s - loss: 0.9354 - acc: 0.9935     
Epoch 191/200
2294/2294 [==============================] - 0s - loss: 0.9404 - acc: 0.9930     
Epoch 192/200
2294/2294 [==============================] - 0s - loss: 0.9326 - acc: 0.9956     
Epoch 193/200
2294/2294 [==============================] - 0s - loss: 0.9311 - acc: 0.9922     
Epoch 194/200
2294/2294 [==============================] - 0s - loss: 0.9270 - acc: 0.9922     
Epoch 195/200
2294/2294 [=============================

2294/2294 [==============================] - 0s - loss: 1.0378 - acc: 0.9935     
Epoch 154/200
2294/2294 [==============================] - 0s - loss: 1.0351 - acc: 0.9913     
Epoch 155/200
2294/2294 [==============================] - 0s - loss: 1.0364 - acc: 0.9882     
Epoch 156/200
2294/2294 [==============================] - 0s - loss: 1.0234 - acc: 0.9961     
Epoch 157/200
2294/2294 [==============================] - 0s - loss: 1.0250 - acc: 0.9922     
Epoch 158/200
2294/2294 [==============================] - 0s - loss: 1.0250 - acc: 0.9904     
Epoch 159/200
2294/2294 [==============================] - 0s - loss: 1.0218 - acc: 0.9895     
Epoch 160/200
2294/2294 [==============================] - 0s - loss: 1.0197 - acc: 0.9917     
Epoch 161/200
2294/2294 [==============================] - 0s - loss: 1.0160 - acc: 0.9926     
Epoch 162/200
2294/2294 [==============================] - 0s - loss: 1.0122 - acc: 0.9922     
Epoch 163/200
2294/2294 [=============================

2294/2294 [==============================] - 0s - loss: 1.1302 - acc: 0.9891     
Epoch 123/200
2294/2294 [==============================] - 0s - loss: 1.1226 - acc: 0.9895     
Epoch 124/200
2294/2294 [==============================] - 0s - loss: 1.1129 - acc: 0.9935     
Epoch 125/200
2294/2294 [==============================] - 0s - loss: 1.1087 - acc: 0.9926     
Epoch 126/200
2294/2294 [==============================] - 0s - loss: 1.1082 - acc: 0.9913     
Epoch 127/200
2294/2294 [==============================] - 0s - loss: 1.0995 - acc: 0.9913     
Epoch 128/200
2294/2294 [==============================] - 0s - loss: 1.1027 - acc: 0.9874     
Epoch 129/200
2294/2294 [==============================] - 0s - loss: 1.0928 - acc: 0.9913     
Epoch 130/200
2294/2294 [==============================] - 0s - loss: 1.0889 - acc: 0.9908     
Epoch 131/200
2294/2294 [==============================] - 0s - loss: 1.0831 - acc: 0.9917     
Epoch 132/200
2294/2294 [=============================

2294/2294 [==============================] - 0s - loss: 1.3751 - acc: 0.9861     
Epoch 92/200
2294/2294 [==============================] - 0s - loss: 1.3681 - acc: 0.9891     
Epoch 93/200
2294/2294 [==============================] - 0s - loss: 1.3599 - acc: 0.9847     
Epoch 94/200
2294/2294 [==============================] - 0s - loss: 1.3474 - acc: 0.9887     
Epoch 95/200
2294/2294 [==============================] - 0s - loss: 1.3432 - acc: 0.9865     
Epoch 96/200
2294/2294 [==============================] - 0s - loss: 1.3334 - acc: 0.9878     
Epoch 97/200
2294/2294 [==============================] - 0s - loss: 1.3294 - acc: 0.9878     
Epoch 98/200
2294/2294 [==============================] - 0s - loss: 1.3167 - acc: 0.9908     
Epoch 99/200
2294/2294 [==============================] - 0s - loss: 1.3125 - acc: 0.9874     
Epoch 100/200
2294/2294 [==============================] - 0s - loss: 1.3043 - acc: 0.9891     
Epoch 101/200
2294/2294 [==============================] - 0s 

2295/2295 [==============================] - 0s - loss: 1.8164 - acc: 0.9821     
Epoch 60/200
2295/2295 [==============================] - 0s - loss: 1.7976 - acc: 0.9821     
Epoch 61/200
2295/2295 [==============================] - 0s - loss: 1.7801 - acc: 0.9821     
Epoch 62/200
2295/2295 [==============================] - 0s - loss: 1.7607 - acc: 0.9839     
Epoch 63/200
2295/2295 [==============================] - 0s - loss: 1.7444 - acc: 0.9865     
Epoch 64/200
2295/2295 [==============================] - 0s - loss: 1.7325 - acc: 0.9821     
Epoch 65/200
2295/2295 [==============================] - 0s - loss: 1.7150 - acc: 0.9813     
Epoch 66/200
2295/2295 [==============================] - 0s - loss: 1.7020 - acc: 0.9830     
Epoch 67/200
2295/2295 [==============================] - 0s - loss: 1.6843 - acc: 0.9834     
Epoch 68/200
2295/2295 [==============================] - 0s - loss: 1.6664 - acc: 0.9847     
Epoch 69/200
2295/2295 [==============================] - 0s - 

In [8]:
# BEST

folds = 10
kfold = StratifiedKFold(n_splits=folds, shuffle=True)
cv_accuracies = []
cv_precision = []
cv_recall = []
cv_f1_score = []

for index, (train_indices, val_indices) in enumerate(kfold.split(tactile_images, labels)):
    print("Training fold " + str(index + 1) + "/" + str(folds))

    # split data
    images_train, images_val = tactile_images[train_indices], tactile_images[val_indices]
    labels_train, labels_val = labels_cat[train_indices], labels_cat[val_indices]  
    
    # build model
    epochs = 200
    batch = 32
    
    learning_rate = 0.0001
    epsilon = 1e-08
    decay_rate = 0.005
    
    l2_reg = 0.05
    drop_prob = 0.4
    
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=(FINGERS, TACTILE_IMAGE_ROWS, TACTILE_IMAGE_COLS), 
                     data_format='channels_first',
                     use_bias=False, kernel_regularizer=regularizers.l2(l2_reg)))
    model.add(BatchNormalization(axis=1))
    model.add(Activation('relu'))
        
    model.add(Flatten())
    
    model.add(Dense(1024, kernel_regularizer=regularizers.l2(l2_reg)))
    model.add(BatchNormalization(axis=1))
    model.add(Activation('relu'))
    model.add(Dropout(drop_prob))
        
    model.add(Dense(2, activation='softmax'))

    # compile and train
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.Adam(lr=learning_rate, epsilon=epsilon, decay=decay_rate), 
                  metrics=['accuracy'])
    model.fit(images_train, labels_train, epochs=epochs, batch_size=batch, verbose=1)
    
    # evaluate
    print("# # # Evaluating cv-fold...")
    scores = model.evaluate(images_val, labels_val, verbose=1)
    cv_accuracies.append(scores[1] * 100)
    
    predictions = model.predict(images_val)
    [precision, recall, f1_score, _] = precision_recall_fscore_support(np.argmax(labels_val, axis=1), 
                                                                          np.argmax(predictions, axis=1), 
                                                                          average='binary', pos_label=1)
    
    cv_precision.append(precision * 100)
    cv_recall.append(recall * 100)
    cv_f1_score.append(f1_score * 100)
    
    print("\naccuracy:", scores[1] * 100)
    print("precision:", precision * 100)
    print("recall:", recall * 100)
    print("f1_score:", f1_score * 100)
    
print("\nFinal cross-validation score:")
print(np.mean(cv_accuracies), "+/-", np.std(cv_accuracies))
print(np.mean(cv_precision), "+/-", np.std(cv_precision))
print(np.mean(cv_recall), "+/-", np.std(cv_recall))
print(np.mean(cv_f1_score), "+/-", np.std(cv_f1_score))

#93.8012967404 +/- 1.42350607745
#92.6639929059 +/- 1.94284221898
#95.2288385827 +/- 1.87036444279
#93.9102458431 +/- 1.38359156875

Training fold 1/10
Epoch 1/200
2294/2294 [==============================] - 1s - loss: 63.3404 - acc: 0.8296     
Epoch 2/200
2294/2294 [==============================] - 0s - loss: 44.8297 - acc: 0.8697     
Epoch 3/200
2294/2294 [==============================] - 0s - loss: 34.1235 - acc: 0.8827     
Epoch 4/200
2294/2294 [==============================] - 0s - loss: 27.2388 - acc: 0.8958     
Epoch 5/200
2294/2294 [==============================] - 0s - loss: 22.5295 - acc: 0.8862     
Epoch 6/200
2294/2294 [==============================] - 0s - loss: 19.0952 - acc: 0.9076     
Epoch 7/200
2294/2294 [==============================] - 0s - loss: 16.5302 - acc: 0.9076     
Epoch 8/200
2294/2294 [==============================] - 0s - loss: 14.5376 - acc: 0.9215     
Epoch 9/200
2294/2294 [==============================] - 0s - loss: 12.9487 - acc: 0.9211     
Epoch 10/200
2294/2294 [==============================] - 0s - loss: 11.6902 - acc: 0.9211     
Epoch 11/200
2294/2294 [======

2294/2294 [==============================] - 0s - loss: 2.3453 - acc: 0.9699     
Epoch 88/200
2294/2294 [==============================] - 0s - loss: 2.3337 - acc: 0.9721     
Epoch 89/200
2294/2294 [==============================] - 0s - loss: 2.3268 - acc: 0.9695     
Epoch 90/200
2294/2294 [==============================] - 0s - loss: 2.3074 - acc: 0.9721     
Epoch 91/200
2294/2294 [==============================] - 0s - loss: 2.2986 - acc: 0.9708     
Epoch 92/200
2294/2294 [==============================] - 0s - loss: 2.2949 - acc: 0.9725     
Epoch 93/200
2294/2294 [==============================] - 0s - loss: 2.2774 - acc: 0.9686     
Epoch 94/200
2294/2294 [==============================] - 0s - loss: 2.2715 - acc: 0.9690     
Epoch 95/200
2294/2294 [==============================] - 0s - loss: 2.2559 - acc: 0.9725     
Epoch 96/200
2294/2294 [==============================] - 0s - loss: 2.2495 - acc: 0.9717     
Epoch 97/200
2294/2294 [==============================] - 0s - 

2294/2294 [==============================] - 0s - loss: 2.9280 - acc: 0.9673     
Epoch 56/200
2294/2294 [==============================] - 0s - loss: 2.9008 - acc: 0.9712     
Epoch 57/200
2294/2294 [==============================] - 0s - loss: 2.8816 - acc: 0.9634     
Epoch 58/200
2294/2294 [==============================] - 0s - loss: 2.8402 - acc: 0.9708     
Epoch 59/200
2294/2294 [==============================] - 0s - loss: 2.8291 - acc: 0.9621     
Epoch 60/200
2294/2294 [==============================] - 0s - loss: 2.8078 - acc: 0.9590     
Epoch 61/200
2294/2294 [==============================] - 0s - loss: 2.7893 - acc: 0.9582     
Epoch 62/200
2294/2294 [==============================] - 0s - loss: 2.7490 - acc: 0.9647     
Epoch 63/200
2294/2294 [==============================] - 0s - loss: 2.7358 - acc: 0.9621     
Epoch 64/200
2294/2294 [==============================] - 0s - loss: 2.7124 - acc: 0.9634     
Epoch 65/200
2294/2294 [==============================] - 0s - 

2294/2294 [==============================] - 0s - loss: 5.3144 - acc: 0.9381     
Epoch 24/200
2294/2294 [==============================] - 0s - loss: 5.1131 - acc: 0.9429     
Epoch 25/200
2294/2294 [==============================] - 0s - loss: 4.9384 - acc: 0.9433     
Epoch 26/200
2294/2294 [==============================] - 0s - loss: 4.7876 - acc: 0.9494     
Epoch 27/200
2294/2294 [==============================] - 0s - loss: 4.6524 - acc: 0.9429     
Epoch 28/200
2294/2294 [==============================] - 0s - loss: 4.5098 - acc: 0.9499     
Epoch 29/200
2294/2294 [==============================] - 0s - loss: 4.3985 - acc: 0.9477     
Epoch 30/200
2294/2294 [==============================] - 0s - loss: 4.2879 - acc: 0.9464     
Epoch 31/200
2294/2294 [==============================] - 0s - loss: 4.1872 - acc: 0.9451     
Epoch 32/200
2294/2294 [==============================] - 0s - loss: 4.0934 - acc: 0.9412     
Epoch 33/200
2294/2294 [==============================] - 0s - 

2294/2294 [==============================] - 0s - loss: 1.6379 - acc: 0.9786     
Epoch 195/200
2294/2294 [==============================] - 0s - loss: 1.6383 - acc: 0.9765     
Epoch 196/200
2294/2294 [==============================] - 0s - loss: 1.6309 - acc: 0.9773     
Epoch 197/200
2294/2294 [==============================] - 0s - loss: 1.6277 - acc: 0.9786     
Epoch 198/200
2294/2294 [==============================] - 0s - loss: 1.6253 - acc: 0.9782     
Epoch 199/200
2294/2294 [==============================] - 0s - loss: 1.6149 - acc: 0.9817     
Epoch 200/200
2294/2294 [==============================] - 0s - loss: 1.6155 - acc: 0.9778     
# # # Evaluating cv-fold...
 32/255 [==>...........................] - ETA: 3s
accuracy: 97.64705847291386
precision: 97.65625
recall: 97.65625
f1_score: 97.65625
Training fold 4/10
Epoch 1/200
2294/2294 [==============================] - 1s - loss: 63.5025 - acc: 0.8073     
Epoch 2/200
2294/2294 [==============================] - 0s - los

...

2294/2294 [==============================] - 0s - loss: 2.5872 - acc: 0.9621     
Epoch 69/200
2294/2294 [==============================] - 0s - loss: 2.5659 - acc: 0.9699     
Epoch 70/200
2294/2294 [==============================] - 0s - loss: 2.5472 - acc: 0.9690     
Epoch 71/200
2294/2294 [==============================] - 0s - loss: 2.5228 - acc: 0.9660     
Epoch 72/200
2294/2294 [==============================] - 0s - loss: 2.5015 - acc: 0.9743     
Epoch 73/200
2294/2294 [==============================] - 0s - loss: 2.4894 - acc: 0.9686     
Epoch 74/200
2294/2294 [==============================] - 0s - loss: 2.4820 - acc: 0.9647     
Epoch 75/200
2294/2294 [==============================] - 0s - loss: 2.4663 - acc: 0.9634     
Epoch 76/200
2294/2294 [==============================] - 0s - loss: 2.4425 - acc: 0.9677     
Epoch 77/200
2294/2294 [==============================] - 0s - loss: 2.4272 - acc: 0.9699     
Epoch 78/200
2294/2294 [==============================] - 0s - 

2294/2294 [==============================] - 0s - loss: 3.7633 - acc: 0.9534     
Epoch 37/200
2294/2294 [==============================] - 0s - loss: 3.6975 - acc: 0.9516     
Epoch 38/200
2294/2294 [==============================] - 0s - loss: 3.6299 - acc: 0.9538     
Epoch 39/200
2294/2294 [==============================] - 0s - loss: 3.5606 - acc: 0.9595     
Epoch 40/200
2294/2294 [==============================] - 0s - loss: 3.5111 - acc: 0.9525     
Epoch 41/200
2294/2294 [==============================] - 0s - loss: 3.4620 - acc: 0.9551     
Epoch 42/200
2294/2294 [==============================] - 1s - loss: 3.4150 - acc: 0.9534     
Epoch 43/200
2294/2294 [==============================] - 0s - loss: 3.3561 - acc: 0.9538     
Epoch 44/200
2294/2294 [==============================] - 0s - loss: 3.3079 - acc: 0.9603     
Epoch 45/200
2294/2294 [==============================] - 0s - loss: 3.2678 - acc: 0.9551     
Epoch 46/200
2294/2294 [==============================] - 0s - 

2294/2294 [==============================] - 1s - loss: 27.0676 - acc: 0.8984     
Epoch 5/200
2294/2294 [==============================] - 1s - loss: 22.3047 - acc: 0.9037     
Epoch 6/200
2294/2294 [==============================] - 0s - loss: 18.8547 - acc: 0.9098     
Epoch 7/200
2294/2294 [==============================] - 0s - loss: 16.2704 - acc: 0.9154     
Epoch 8/200
2294/2294 [==============================] - 0s - loss: 14.2834 - acc: 0.9133     
Epoch 9/200
2294/2294 [==============================] - 0s - loss: 12.6986 - acc: 0.9172     
Epoch 10/200
2294/2294 [==============================] - 0s - loss: 11.4174 - acc: 0.9250     
Epoch 11/200
2294/2294 [==============================] - 0s - loss: 10.3857 - acc: 0.9215     
Epoch 12/200
2294/2294 [==============================] - 0s - loss: 9.4968 - acc: 0.9307     
Epoch 13/200
2294/2294 [==============================] - 0s - loss: 8.7659 - acc: 0.9346     
Epoch 14/200
2294/2294 [==============================] - 0s

2294/2294 [==============================] - 0s - loss: 1.6666 - acc: 0.9799     
Epoch 175/200
2294/2294 [==============================] - 0s - loss: 1.6711 - acc: 0.9738     
Epoch 176/200
2294/2294 [==============================] - 0s - loss: 1.6644 - acc: 0.9795     
Epoch 177/200
2294/2294 [==============================] - 0s - loss: 1.6605 - acc: 0.9773     
Epoch 178/200
2294/2294 [==============================] - 0s - loss: 1.6608 - acc: 0.9752     
Epoch 179/200
2294/2294 [==============================] - 0s - loss: 1.6472 - acc: 0.9813     
Epoch 180/200
2294/2294 [==============================] - 0s - loss: 1.6422 - acc: 0.9773     
Epoch 181/200
2294/2294 [==============================] - 0s - loss: 1.6500 - acc: 0.9773     
Epoch 182/200
2294/2294 [==============================] - 0s - loss: 1.6462 - acc: 0.9760     
Epoch 183/200
2294/2294 [==============================] - 0s - loss: 1.6348 - acc: 0.9795     
Epoch 184/200
2294/2294 [=============================

2295/2295 [==============================] - 0s - loss: 1.7387 - acc: 0.9773     
Epoch 143/200
2295/2295 [==============================] - 0s - loss: 1.7386 - acc: 0.9699     
Epoch 144/200
2295/2295 [==============================] - 0s - loss: 1.7284 - acc: 0.9773     
Epoch 145/200
2295/2295 [==============================] - 0s - loss: 1.7223 - acc: 0.9756     
Epoch 146/200
2295/2295 [==============================] - 0s - loss: 1.7174 - acc: 0.9795     
Epoch 147/200
2295/2295 [==============================] - 0s - loss: 1.7103 - acc: 0.9782     
Epoch 148/200
2295/2295 [==============================] - 0s - loss: 1.7045 - acc: 0.9786     
Epoch 149/200
2295/2295 [==============================] - 0s - loss: 1.7084 - acc: 0.9778     
Epoch 150/200
2295/2295 [==============================] - 0s - loss: 1.7052 - acc: 0.9760     
Epoch 151/200
2295/2295 [==============================] - 0s - loss: 1.6944 - acc: 0.9765     
Epoch 152/200
2295/2295 [=============================

In [9]:
# BEST

folds = 10
kfold = StratifiedKFold(n_splits=folds, shuffle=True)
cv_accuracies = []
cv_precision = []
cv_recall = []
cv_f1_score = []

for index, (train_indices, val_indices) in enumerate(kfold.split(tactile_images, labels)):
    print("Training fold " + str(index + 1) + "/" + str(folds))

    # split data
    images_train, images_val = tactile_images[train_indices], tactile_images[val_indices]
    labels_train, labels_val = labels_cat[train_indices], labels_cat[val_indices]  
    
    # build model
    epochs = 300
    batch = 32
    
    learning_rate = 0.0001
    epsilon = 1e-08
    decay_rate = 0.004
    
    l2_reg = 0.01
    drop_prob = 0.0
    
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=(FINGERS, TACTILE_IMAGE_ROWS, TACTILE_IMAGE_COLS), 
                     data_format='channels_first',
                     use_bias=False, kernel_regularizer=regularizers.l2(l2_reg)))
    model.add(BatchNormalization(axis=1))
    model.add(Activation('relu'))
    
    model.add(Flatten())
    
    model.add(Dense(1024, kernel_regularizer=regularizers.l2(l2_reg)))
    model.add(BatchNormalization(axis=1))
    model.add(Activation('relu'))
    model.add(Dropout(drop_prob))
    
    model.add(Dense(2, activation='softmax'))

    # compile and train
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.Adam(lr=learning_rate, epsilon=epsilon, decay=decay_rate), 
                  metrics=['accuracy'])
    model.fit(images_train, labels_train, epochs=epochs, batch_size=batch, verbose=1)
    
    # evaluate
    print("# # # Evaluating cv-fold...")
    scores = model.evaluate(images_val, labels_val, verbose=1)
    cv_accuracies.append(scores[1] * 100)
    
    predictions = model.predict(images_val)
    [precision, recall, f1_score, _] = precision_recall_fscore_support(np.argmax(labels_val, axis=1), 
                                                                          np.argmax(predictions, axis=1), 
                                                                          average='binary', pos_label=1)
    
    cv_precision.append(precision * 100)
    cv_recall.append(recall * 100)
    cv_f1_score.append(f1_score * 100)
    
    print("\naccuracy:", scores[1] * 100)
    print("precision:", precision * 100)
    print("recall:", recall * 100)
    print("f1_score:", f1_score * 100)
    
print("\nFinal cross-validation score:")
print(np.mean(cv_accuracies), "+/-", np.std(cv_accuracies))
print(np.mean(cv_precision), "+/-", np.std(cv_precision))
print(np.mean(cv_recall), "+/-", np.std(cv_recall))
print(np.mean(cv_f1_score), "+/-", np.std(cv_f1_score))

#93.95754192005029 +/- 1.3993579451263178
#92.98521156585969 +/- 1.5375542848477914
#95.15071358267717 +/- 1.8815213772472403
#94.04503371894552 +/- 1.3907299313722477

Training fold 1/10
Epoch 1/300
2294/2294 [==============================] - 1s - loss: 13.5312 - acc: 0.8426     
Epoch 2/300
2294/2294 [==============================] - 0s - loss: 10.4203 - acc: 0.8963     
Epoch 3/300
2294/2294 [==============================] - 0s - loss: 8.5159 - acc: 0.9202     
Epoch 4/300
2294/2294 [==============================] - 0s - loss: 7.2291 - acc: 0.9246     
Epoch 5/300
2294/2294 [==============================] - 0s - loss: 6.3160 - acc: 0.9377     
Epoch 6/300
2294/2294 [==============================] - 0s - loss: 5.6229 - acc: 0.9416     
Epoch 7/300
2294/2294 [==============================] - 0s - loss: 5.0961 - acc: 0.9394     
Epoch 8/300
2294/2294 [==============================] - 0s - loss: 4.6652 - acc: 0.9529     
Epoch 9/300
2294/2294 [==============================] - 0s - loss: 4.3299 - acc: 0.9512     
Epoch 10/300
2294/2294 [==============================] - 0s - loss: 4.0497 - acc: 0.9564     
Epoch 11/300
2294/2294 [==============

2294/2294 [==============================] - 0s - loss: 1.3614 - acc: 0.9843     
Epoch 88/300
2294/2294 [==============================] - 0s - loss: 1.3482 - acc: 0.9908     
Epoch 89/300
2294/2294 [==============================] - 0s - loss: 1.3475 - acc: 0.9895     
Epoch 90/300
2294/2294 [==============================] - 0s - loss: 1.3401 - acc: 0.9887     
Epoch 91/300
2294/2294 [==============================] - 0s - loss: 1.3325 - acc: 0.9913     
Epoch 92/300
2294/2294 [==============================] - 0s - loss: 1.3319 - acc: 0.9856     
Epoch 93/300
2294/2294 [==============================] - 0s - loss: 1.3185 - acc: 0.9904     
Epoch 94/300
2294/2294 [==============================] - 0s - loss: 1.3144 - acc: 0.9900     
Epoch 95/300
2294/2294 [==============================] - 0s - loss: 1.3117 - acc: 0.9891     
Epoch 96/300
2294/2294 [==============================] - 0s - loss: 1.3113 - acc: 0.9861     
Epoch 97/300
2294/2294 [==============================] - 0s - 

2294/2294 [==============================] - 0s - loss: 0.9013 - acc: 0.9943     
Epoch 257/300
2294/2294 [==============================] - 0s - loss: 0.8974 - acc: 0.9948     
Epoch 258/300
2294/2294 [==============================] - 0s - loss: 0.8986 - acc: 0.9943     
Epoch 259/300
2294/2294 [==============================] - 0s - loss: 0.8940 - acc: 0.9965     
Epoch 260/300
2294/2294 [==============================] - 0s - loss: 0.8943 - acc: 0.9974     
Epoch 261/300
2294/2294 [==============================] - 0s - loss: 0.8954 - acc: 0.9965     
Epoch 262/300
2294/2294 [==============================] - 0s - loss: 0.8907 - acc: 0.9965     
Epoch 263/300
2294/2294 [==============================] - 0s - loss: 0.8903 - acc: 0.9965     
Epoch 264/300
2294/2294 [==============================] - 0s - loss: 0.8915 - acc: 0.9939     
Epoch 265/300
2294/2294 [==============================] - 0s - loss: 0.8867 - acc: 0.9956     
Epoch 266/300
2294/2294 [=============================

2294/2294 [==============================] - 0s - loss: 1.2000 - acc: 0.9895     
Epoch 125/300
2294/2294 [==============================] - 0s - loss: 1.1954 - acc: 0.9895     
Epoch 126/300
2294/2294 [==============================] - 0s - loss: 1.1882 - acc: 0.9922     
Epoch 127/300
2294/2294 [==============================] - 0s - loss: 1.1848 - acc: 0.9900     
Epoch 128/300
2294/2294 [==============================] - 0s - loss: 1.1792 - acc: 0.9913     
Epoch 129/300
2294/2294 [==============================] - 0s - loss: 1.1748 - acc: 0.9913     
Epoch 130/300
2294/2294 [==============================] - 0s - loss: 1.1715 - acc: 0.9930     
Epoch 131/300
2294/2294 [==============================] - 0s - loss: 1.1705 - acc: 0.9917     
Epoch 132/300
2294/2294 [==============================] - 0s - loss: 1.1653 - acc: 0.9913     
Epoch 133/300
2294/2294 [==============================] - 0s - loss: 1.1635 - acc: 0.9891     
Epoch 134/300
2294/2294 [=============================

2294/2294 [==============================] - 0s - loss: 0.8623 - acc: 0.9961     
Epoch 295/300
2294/2294 [==============================] - 0s - loss: 0.8635 - acc: 0.9965     
Epoch 296/300
2294/2294 [==============================] - 0s - loss: 0.8679 - acc: 0.9939     
Epoch 297/300
2294/2294 [==============================] - 0s - loss: 0.8623 - acc: 0.9961     
Epoch 298/300
2294/2294 [==============================] - 0s - loss: 0.8632 - acc: 0.9939     
Epoch 299/300
2294/2294 [==============================] - 0s - loss: 0.8590 - acc: 0.9965     
Epoch 300/300
2294/2294 [==============================] - 0s - loss: 0.8591 - acc: 0.9952     
# # # Evaluating cv-fold...
 32/255 [==>...........................] - ETA: 5s
accuracy: 95.2941176704332
precision: 93.93939393939394
recall: 96.875
f1_score: 95.38461538461539
Training fold 3/10
Epoch 1/300
2294/2294 [==============================] - 2s - loss: 13.5514 - acc: 0.8474     
Epoch 2/300
2294/2294 [===========================

...

2294/2294 [==============================] - 0s - loss: 0.9977 - acc: 0.9935     
Epoch 184/300
2294/2294 [==============================] - 0s - loss: 0.9962 - acc: 0.9939     
Epoch 185/300
2294/2294 [==============================] - 0s - loss: 0.9955 - acc: 0.9922     
Epoch 186/300
2294/2294 [==============================] - 0s - loss: 0.9899 - acc: 0.9943     
Epoch 187/300
2294/2294 [==============================] - 0s - loss: 0.9916 - acc: 0.9926     
Epoch 188/300
2294/2294 [==============================] - 0s - loss: 0.9840 - acc: 0.9956     
Epoch 189/300
2294/2294 [==============================] - 0s - loss: 0.9894 - acc: 0.9943     
Epoch 190/300
2294/2294 [==============================] - 0s - loss: 0.9853 - acc: 0.9935     
Epoch 191/300
2294/2294 [==============================] - 0s - loss: 0.9797 - acc: 0.9952     
Epoch 192/300
2294/2294 [==============================] - 0s - loss: 0.9762 - acc: 0.9969     
Epoch 193/300
2294/2294 [=============================

2294/2294 [==============================] - 0s - loss: 1.6347 - acc: 0.9830     
Epoch 53/300
2294/2294 [==============================] - 0s - loss: 1.6235 - acc: 0.9826     
Epoch 54/300
2294/2294 [==============================] - 0s - loss: 1.6059 - acc: 0.9847     
Epoch 55/300
2294/2294 [==============================] - 0s - loss: 1.5913 - acc: 0.9869     
Epoch 56/300
2294/2294 [==============================] - 0s - loss: 1.5804 - acc: 0.9839     
Epoch 57/300
2294/2294 [==============================] - 0s - loss: 1.5677 - acc: 0.9847     
Epoch 58/300
2294/2294 [==============================] - 0s - loss: 1.5533 - acc: 0.9852     
Epoch 59/300
2294/2294 [==============================] - 0s - loss: 1.5462 - acc: 0.9847     
Epoch 60/300
2294/2294 [==============================] - 0s - loss: 1.5293 - acc: 0.9878     
Epoch 61/300
2294/2294 [==============================] - 0s - loss: 1.5231 - acc: 0.9834     
Epoch 62/300
2294/2294 [==============================] - 0s - 

2294/2294 [==============================] - 0s - loss: 0.9001 - acc: 0.9961     
Epoch 223/300
2294/2294 [==============================] - 0s - loss: 0.9006 - acc: 0.9943     
Epoch 224/300
2294/2294 [==============================] - 0s - loss: 0.8977 - acc: 0.9948     
Epoch 225/300
2294/2294 [==============================] - 0s - loss: 0.8967 - acc: 0.9961     
Epoch 226/300
2294/2294 [==============================] - 0s - loss: 0.8989 - acc: 0.9913     
Epoch 227/300
2294/2294 [==============================] - 0s - loss: 0.8921 - acc: 0.9969     
Epoch 228/300
2294/2294 [==============================] - 0s - loss: 0.8902 - acc: 0.9956     
Epoch 229/300
2294/2294 [==============================] - 0s - loss: 0.8880 - acc: 0.9961     
Epoch 230/300
2294/2294 [==============================] - 0s - loss: 0.8899 - acc: 0.9943     
Epoch 231/300
2294/2294 [==============================] - 0s - loss: 0.8881 - acc: 0.9952     
Epoch 232/300
2294/2294 [=============================

2295/2295 [==============================] - 0s - loss: 1.3412 - acc: 0.9895     
Epoch 92/300
2295/2295 [==============================] - 0s - loss: 1.3379 - acc: 0.9895     
Epoch 93/300
2295/2295 [==============================] - 0s - loss: 1.3339 - acc: 0.9891     
Epoch 94/300
2295/2295 [==============================] - 0s - loss: 1.3287 - acc: 0.9900     
Epoch 95/300
2295/2295 [==============================] - 0s - loss: 1.3232 - acc: 0.9895     
Epoch 96/300
2295/2295 [==============================] - 0s - loss: 1.3114 - acc: 0.9904     
Epoch 97/300
2295/2295 [==============================] - 0s - loss: 1.3061 - acc: 0.9900     
Epoch 98/300
2295/2295 [==============================] - 0s - loss: 1.3046 - acc: 0.9900     
Epoch 99/300
2295/2295 [==============================] - 0s - loss: 1.2973 - acc: 0.9904     
Epoch 100/300
2295/2295 [==============================] - 0s - loss: 1.2923 - acc: 0.9908     
Epoch 101/300
2295/2295 [==============================] - 0s 

2295/2295 [==============================] - 0s - loss: 0.8934 - acc: 0.9956     
Epoch 262/300
2295/2295 [==============================] - 0s - loss: 0.8901 - acc: 0.9961     
Epoch 263/300
2295/2295 [==============================] - 0s - loss: 0.8942 - acc: 0.9952     
Epoch 264/300
2295/2295 [==============================] - 0s - loss: 0.8897 - acc: 0.9948     
Epoch 265/300
2295/2295 [==============================] - 0s - loss: 0.8858 - acc: 0.9965     
Epoch 266/300
2295/2295 [==============================] - 0s - loss: 0.8851 - acc: 0.9956     
Epoch 267/300
2295/2295 [==============================] - 0s - loss: 0.8871 - acc: 0.9943     
Epoch 268/300
2295/2295 [==============================] - 0s - loss: 0.8826 - acc: 0.9952     
Epoch 269/300
2295/2295 [==============================] - 0s - loss: 0.8847 - acc: 0.9935     
Epoch 270/300
2295/2295 [==============================] - 0s - loss: 0.8786 - acc: 0.9969     
Epoch 271/300
2295/2295 [=============================

In [5]:
# BEST RESULT (REPORTED FOR IROS)

folds = 10
kfold = StratifiedKFold(n_splits=folds, shuffle=True)
cv_accuracies = []
cv_precision = []
cv_recall = []
cv_f1_score = []

for index, (train_indices, val_indices) in enumerate(kfold.split(tactile_images, labels)):
    print("Training fold " + str(index + 1) + "/" + str(folds))

    # split data
    images_train, images_val = tactile_images[train_indices], tactile_images[val_indices]
    labels_train, labels_val = labels_cat[train_indices], labels_cat[val_indices]  
    
    # build model
    epochs = 200
    batch = 32
    
    learning_rate = 0.0001
    epsilon = 1e-08
    decay_rate = 0.005
    
    l2_reg = 0.01
    drop_prob = 0.0
    
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=(FINGERS, TACTILE_IMAGE_ROWS, TACTILE_IMAGE_COLS), 
                     data_format='channels_first',
                     use_bias=False, kernel_regularizer=regularizers.l2(l2_reg)))
    model.add(BatchNormalization(axis=1))
    model.add(Activation('relu'))
    
    model.add(Flatten())
    
    model.add(Dense(1024, kernel_regularizer=regularizers.l2(l2_reg)))
    model.add(BatchNormalization(axis=1))
    model.add(Activation('relu'))
    model.add(Dropout(drop_prob))
    
    model.add(Dense(2, activation='softmax'))

    # compile and train
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.Adam(lr=learning_rate, epsilon=epsilon, decay=decay_rate), 
                  metrics=['accuracy'])
    model.fit(images_train, labels_train, epochs=epochs, batch_size=batch, verbose=1)
    
    # evaluate
    print("# # # Evaluating cv-fold...")
    scores = model.evaluate(images_val, labels_val, verbose=1)
    cv_accuracies.append(scores[1] * 100)
    
    predictions = model.predict(images_val)
    [precision, recall, f1_score, _] = precision_recall_fscore_support(np.argmax(labels_val, axis=1), 
                                                                          np.argmax(predictions, axis=1), 
                                                                          average='binary', pos_label=1)
    
    cv_precision.append(precision * 100)
    cv_recall.append(recall * 100)
    cv_f1_score.append(f1_score * 100)
    
    print("\naccuracy:", scores[1] * 100)
    print("precision:", precision * 100)
    print("recall:", recall * 100)
    print("f1_score:", f1_score * 100)
    
print("\nFinal cross-validation score:")
print(np.mean(cv_accuracies), "+/-", np.std(cv_accuracies))
print(np.mean(cv_precision), "+/-", np.std(cv_precision))
print(np.mean(cv_recall), "+/-", np.std(cv_recall))
print(np.mean(cv_f1_score), "+/-", np.std(cv_f1_score))

# IROS
#94.0370540846 +/- 1.47500222339
#92.9833007648 +/- 2.40039058083
#95.389394685 +/- 1.61583557159
#94.1460645818 +/- 1.39118951759

# cero2lesscontact - D7
#94.1551643508589 +/- 1.5417496158345507
#93.10975493048241 +/- 2.071176554226488
#95.46567421259843 +/- 2.030324351541223
#94.2518574823022 +/- 1.4971527138687257

Training fold 1/10
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Epoch 1/200
2294/2294 [==============================] - 3s - loss: 13.8467 - acc: 0.8609     
Epoch 2/200
2294/2294 [==============================] - 0s - loss: 11.1956 - acc: 0.9093     
Epoch 3/200
2294/2294 [==============================] - 0s - loss: 9.5233 - acc: 0.9285     
Epoch 4/200
2294/2294 [==============================] - 0s - loss: 8.3390 - acc: 0.9398     
Epoch 5/200
2294/2294 [==============================] - 0s - loss: 7.4806 - acc: 0.9420     
Epoch 6/200
2294/2294 [==============================] - 0s - loss: 6.8049 - acc: 0.9468     
Epoch 7/200
2294/2294 [==============================] - 0s - loss: 6.2641 - acc: 0.9590     
Epoch 8/200
2294/2294 [==============================] - 0s - loss: 5.8367 - acc: 0.9547     
Epoch 9/

2294/2294 [==============================] - 0s - loss: 1.8675 - acc: 0.9913     
Epoch 75/200
2294/2294 [==============================] - 0s - loss: 1.8558 - acc: 0.9913     
Epoch 76/200
2294/2294 [==============================] - 0s - loss: 1.8445 - acc: 0.9917     
Epoch 77/200
2294/2294 [==============================] - 0s - loss: 1.8301 - acc: 0.9943     
Epoch 78/200
2294/2294 [==============================] - 0s - loss: 1.8200 - acc: 0.9922     
Epoch 79/200
2294/2294 [==============================] - 0s - loss: 1.8150 - acc: 0.9913     
Epoch 80/200
2294/2294 [==============================] - 0s - loss: 1.8017 - acc: 0.9917     
Epoch 81/200
2294/2294 [==============================] - 0s - loss: 1.7949 - acc: 0.9904     
Epoch 82/200
2294/2294 [==============================] - 0s - loss: 1.7853 - acc: 0.9904     
Epoch 83/200
2294/2294 [==============================] - 0s - loss: 1.7734 - acc: 0.9935     
Epoch 84/200
2294/2294 [==============================] - 0s - 

2294/2294 [==============================] - 0s - loss: 1.3367 - acc: 0.9961     
Epoch 160/200
2294/2294 [==============================] - 0s - loss: 1.3342 - acc: 0.9956     
Epoch 161/200
2294/2294 [==============================] - 0s - loss: 1.3307 - acc: 0.9974     
Epoch 162/200
2294/2294 [==============================] - 0s - loss: 1.3240 - acc: 0.9939     
Epoch 163/200
2294/2294 [==============================] - 0s - loss: 1.3213 - acc: 0.9948     
Epoch 164/200
2294/2294 [==============================] - 0s - loss: 1.3159 - acc: 0.9965     
Epoch 165/200
2294/2294 [==============================] - 0s - loss: 1.3135 - acc: 0.9965     
Epoch 166/200
2294/2294 [==============================] - 0s - loss: 1.3126 - acc: 0.9952     - ETA: 0s - loss: 1.3101 - a
Epoch 167/200
2294/2294 [==============================] - 0s - loss: 1.3069 - acc: 0.9969     
Epoch 168/200
2294/2294 [==============================] - 0s - loss: 1.3050 - acc: 0.9969     
Epoch 169/200
2294/2294 [=

2294/2294 [==============================] - 0s - loss: 2.5186 - acc: 0.9791     
Epoch 43/200
2294/2294 [==============================] - 0s - loss: 2.4752 - acc: 0.9878     
Epoch 44/200
2294/2294 [==============================] - 0s - loss: 2.4534 - acc: 0.9839     
Epoch 45/200
2294/2294 [==============================] - 0s - loss: 2.4301 - acc: 0.9856     
Epoch 46/200
2294/2294 [==============================] - 0s - loss: 2.4062 - acc: 0.9843     
Epoch 47/200
2294/2294 [==============================] - 0s - loss: 2.3788 - acc: 0.9878     
Epoch 48/200
2294/2294 [==============================] - 0s - loss: 2.3548 - acc: 0.9861     
Epoch 49/200
2294/2294 [==============================] - 0s - loss: 2.3323 - acc: 0.9882     
Epoch 50/200
2294/2294 [==============================] - 0s - loss: 2.3101 - acc: 0.9887     
Epoch 51/200
2294/2294 [==============================] - 0s - loss: 2.2896 - acc: 0.9887     
Epoch 52/200
2294/2294 [==============================] - 0s - 

2294/2294 [==============================] - 0s - loss: 1.5339 - acc: 0.9943     
Epoch 129/200
2294/2294 [==============================] - 0s - loss: 1.5307 - acc: 0.9952     
Epoch 130/200
2294/2294 [==============================] - 0s - loss: 1.5257 - acc: 0.9939     
Epoch 131/200
2294/2294 [==============================] - 0s - loss: 1.5203 - acc: 0.9948     
Epoch 132/200
2294/2294 [==============================] - 0s - loss: 1.5131 - acc: 0.9969     
Epoch 133/200
2294/2294 [==============================] - 0s - loss: 1.5087 - acc: 0.9952     
Epoch 134/200
2294/2294 [==============================] - 0s - loss: 1.5046 - acc: 0.9952     
Epoch 135/200
2294/2294 [==============================] - 0s - loss: 1.5002 - acc: 0.9948     
Epoch 136/200
2294/2294 [==============================] - 0s - loss: 1.4950 - acc: 0.9965     
Epoch 137/200
2294/2294 [==============================] - 0s - loss: 1.4887 - acc: 0.9948     
Epoch 138/200
2294/2294 [=============================

...

2294/2294 [==============================] - 0s - loss: 1.3041 - acc: 0.9974     
Epoch 198/200
2294/2294 [==============================] - 0s - loss: 1.3063 - acc: 0.9952     
Epoch 199/200
2294/2294 [==============================] - 0s - loss: 1.3030 - acc: 0.9956     
Epoch 200/200
2294/2294 [==============================] - 0s - loss: 1.2999 - acc: 0.9965     
# # # Evaluating cv-fold...
 32/255 [==>...........................] - ETA: 1s
accuracy: 93.33333300609215
precision: 90.51094890510949
recall: 96.875
f1_score: 93.58490566037736
Training fold 9/10
Epoch 1/200
2294/2294 [==============================] - 1s - loss: 13.9715 - acc: 0.8514     
Epoch 2/200
2294/2294 [==============================] - 0s - loss: 11.3855 - acc: 0.9128     
Epoch 3/200
2294/2294 [==============================] - 0s - loss: 9.7402 - acc: 0.9207     
Epoch 4/200
2294/2294 [==============================] - 0s - loss: 8.5774 - acc: 0.9385     
Epoch 5/200
2294/2294 [==============================]

2294/2294 [==============================] - 0s - loss: 1.9030 - acc: 0.9926     
Epoch 82/200
2294/2294 [==============================] - 0s - loss: 1.8911 - acc: 0.9935     
Epoch 83/200
2294/2294 [==============================] - 0s - loss: 1.8824 - acc: 0.9939     
Epoch 84/200
2294/2294 [==============================] - 0s - loss: 1.8701 - acc: 0.9935     
Epoch 85/200
2294/2294 [==============================] - 0s - loss: 1.8609 - acc: 0.9926     
Epoch 86/200
2294/2294 [==============================] - 0s - loss: 1.8560 - acc: 0.9913     
Epoch 87/200
2294/2294 [==============================] - 0s - loss: 1.8426 - acc: 0.9930     
Epoch 88/200
2294/2294 [==============================] - 0s - loss: 1.8342 - acc: 0.9969     
Epoch 89/200
2294/2294 [==============================] - ETA: 0s - loss: 1.8241 - acc: 0.995 - 0s - loss: 1.8238 - acc: 0.9952     
Epoch 90/200
2294/2294 [==============================] - 0s - loss: 1.8158 - acc: 0.9939     
Epoch 91/200
2294/2294 [=

2294/2294 [==============================] - 0s - loss: 1.3792 - acc: 0.9974     
Epoch 167/200
2294/2294 [==============================] - 0s - loss: 1.3763 - acc: 0.9978     
Epoch 168/200
2294/2294 [==============================] - 0s - loss: 1.3744 - acc: 0.9974     
Epoch 169/200
2294/2294 [==============================] - 0s - loss: 1.3691 - acc: 0.9978     
Epoch 170/200
2294/2294 [==============================] - 0s - loss: 1.3667 - acc: 0.9961     
Epoch 171/200
2294/2294 [==============================] - 0s - loss: 1.3623 - acc: 0.9969     
Epoch 172/200
2294/2294 [==============================] - 0s - loss: 1.3615 - acc: 0.9969     
Epoch 173/200
2294/2294 [==============================] - 0s - loss: 1.3565 - acc: 0.9974     
Epoch 174/200
2294/2294 [==============================] - 0s - loss: 1.3545 - acc: 0.9948     
Epoch 175/200
2294/2294 [==============================] - 0s - loss: 1.3481 - acc: 0.9974     
Epoch 176/200
2294/2294 [=============================

2295/2295 [==============================] - 0s - loss: 2.3609 - acc: 0.9900     
Epoch 50/200
2295/2295 [==============================] - 0s - loss: 2.3322 - acc: 0.9930     
Epoch 51/200
2295/2295 [==============================] - 0s - loss: 2.3122 - acc: 0.9878     
Epoch 52/200
2295/2295 [==============================] - 0s - loss: 2.2935 - acc: 0.9904     
Epoch 53/200
2295/2295 [==============================] - 0s - loss: 2.2727 - acc: 0.9882     
Epoch 54/200
2295/2295 [==============================] - 0s - loss: 2.2496 - acc: 0.9917     
Epoch 55/200
2295/2295 [==============================] - 0s - loss: 2.2338 - acc: 0.9922     
Epoch 56/200
2295/2295 [==============================] - 0s - loss: 2.2149 - acc: 0.9878     
Epoch 57/200
2295/2295 [==============================] - 0s - loss: 2.1965 - acc: 0.9917     
Epoch 58/200
2295/2295 [==============================] - 0s - loss: 2.1817 - acc: 0.9900     
Epoch 59/200
2295/2295 [==============================] - 0s - 

2295/2295 [==============================] - 0s - loss: 1.4901 - acc: 0.9939     
Epoch 136/200
2295/2295 [==============================] - 0s - loss: 1.4822 - acc: 0.9965     
Epoch 137/200
2295/2295 [==============================] - 0s - loss: 1.4824 - acc: 0.9948     
Epoch 138/200
2295/2295 [==============================] - 0s - loss: 1.4702 - acc: 0.9969     
Epoch 139/200
2295/2295 [==============================] - 0s - loss: 1.4694 - acc: 0.9952     
Epoch 140/200
2295/2295 [==============================] - 0s - loss: 1.4636 - acc: 0.9952     
Epoch 141/200
2295/2295 [==============================] - 0s - loss: 1.4599 - acc: 0.9961     
Epoch 142/200
2295/2295 [==============================] - 0s - loss: 1.4544 - acc: 0.9969     
Epoch 143/200
2295/2295 [==============================] - 0s - loss: 1.4509 - acc: 0.9969     
Epoch 144/200
2295/2295 [==============================] - 0s - loss: 1.4460 - acc: 0.9956     
Epoch 145/200
2295/2295 [=============================

In [6]:
# BEST RESULT (REPORTED FOR IROS)

folds = 10
kfold = StratifiedKFold(n_splits=folds, shuffle=True)
cv_accuracies = []
cv_precision = []
cv_recall = []
cv_f1_score = []

for index, (train_indices, val_indices) in enumerate(kfold.split(tactile_images, labels)):
    print("Training fold " + str(index + 1) + "/" + str(folds))

    # split data
    images_train, images_val = tactile_images[train_indices], tactile_images[val_indices]
    labels_train, labels_val = labels_cat[train_indices], labels_cat[val_indices]  
    
    # build model
    epochs = 300
    batch = 32
    
    learning_rate = 0.0001
    epsilon = 1e-08
    decay_rate = 0.005
    
    l2_reg = 0.01
    drop_prob = 0.0
    
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=(FINGERS, TACTILE_IMAGE_ROWS, TACTILE_IMAGE_COLS), 
                     data_format='channels_first',
                     use_bias=False, kernel_regularizer=regularizers.l2(l2_reg)))
    model.add(BatchNormalization(axis=1))
    model.add(Activation('relu'))
    
    model.add(Flatten())
    
    model.add(Dense(1024, kernel_regularizer=regularizers.l2(l2_reg)))
    model.add(BatchNormalization(axis=1))
    model.add(Activation('relu'))
    model.add(Dropout(drop_prob))
    
    model.add(Dense(2, activation='softmax'))

    # compile and train
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.Adam(lr=learning_rate, epsilon=epsilon, decay=decay_rate), 
                  metrics=['accuracy'])
    model.fit(images_train, labels_train, epochs=epochs, batch_size=batch, verbose=1)
    
    # evaluate
    print("# # # Evaluating cv-fold...")
    scores = model.evaluate(images_val, labels_val, verbose=1)
    cv_accuracies.append(scores[1] * 100)
    
    predictions = model.predict(images_val)
    [precision, recall, f1_score, _] = precision_recall_fscore_support(np.argmax(labels_val, axis=1), 
                                                                          np.argmax(predictions, axis=1), 
                                                                          average='binary', pos_label=1)
    
    cv_precision.append(precision * 100)
    cv_recall.append(recall * 100)
    cv_f1_score.append(f1_score * 100)
    
    print("\naccuracy:", scores[1] * 100)
    print("precision:", precision * 100)
    print("recall:", recall * 100)
    print("f1_score:", f1_score * 100)
    
print("\nFinal cross-validation score:")
print(np.mean(cv_accuracies), "+/-", np.std(cv_accuracies))
print(np.mean(cv_precision), "+/-", np.std(cv_precision))
print(np.mean(cv_recall), "+/-", np.std(cv_recall))
print(np.mean(cv_f1_score), "+/-", np.std(cv_f1_score))

Training fold 1/10
Epoch 1/300
2294/2294 [==============================] - 1s - loss: 13.9656 - acc: 0.8588     
Epoch 2/300
2294/2294 [==============================] - 0s - loss: 11.3603 - acc: 0.9150     
Epoch 3/300
2294/2294 [==============================] - 0s - loss: 9.6926 - acc: 0.9316     
Epoch 4/300
2294/2294 [==============================] - 0s - loss: 8.5328 - acc: 0.9329     
Epoch 5/300
2294/2294 [==============================] - 0s - loss: 7.6506 - acc: 0.9473     
Epoch 6/300
2294/2294 [==============================] - 0s - loss: 6.9847 - acc: 0.9520     
Epoch 7/300
2294/2294 [==============================] - 0s - loss: 6.4424 - acc: 0.9534     
Epoch 8/300
2294/2294 [==============================] - 0s - loss: 6.0136 - acc: 0.9525     
Epoch 9/300
2294/2294 [==============================] - 0s - loss: 5.6389 - acc: 0.9599     
Epoch 10/300
2294/2294 [==============================] - 0s - loss: 5.3294 - acc: 0.9590     
Epoch 11/300
2294/2294 [==============

2294/2294 [==============================] - 0s - loss: 1.8070 - acc: 0.9930     
Epoch 88/300
2294/2294 [==============================] - 0s - loss: 1.7939 - acc: 0.9943     
Epoch 89/300
2294/2294 [==============================] - 0s - loss: 1.7866 - acc: 0.9956     
Epoch 90/300
2294/2294 [==============================] - 0s - loss: 1.7750 - acc: 0.9956     
Epoch 91/300
2294/2294 [==============================] - 0s - loss: 1.7687 - acc: 0.9943     
Epoch 92/300
2294/2294 [==============================] - 0s - loss: 1.7615 - acc: 0.9948     
Epoch 93/300
2294/2294 [==============================] - 0s - loss: 1.7514 - acc: 0.9939     
Epoch 94/300
2294/2294 [==============================] - 0s - loss: 1.7429 - acc: 0.9956     
Epoch 95/300
2294/2294 [==============================] - 0s - loss: 1.7360 - acc: 0.9952     
Epoch 96/300
2294/2294 [==============================] - 0s - loss: 1.7223 - acc: 0.9974     
Epoch 97/300
2294/2294 [==============================] - 0s - 

2294/2294 [==============================] - 0s - loss: 1.3265 - acc: 0.9983     
Epoch 173/300
2294/2294 [==============================] - 0s - loss: 1.3254 - acc: 0.9974     
Epoch 174/300
2294/2294 [==============================] - 0s - loss: 1.3215 - acc: 0.9974     
Epoch 175/300
2294/2294 [==============================] - 0s - loss: 1.3178 - acc: 0.9965     
Epoch 176/300
2294/2294 [==============================] - 0s - loss: 1.3142 - acc: 0.9969     
Epoch 177/300
2294/2294 [==============================] - 0s - loss: 1.3150 - acc: 0.9965     
Epoch 178/300
2294/2294 [==============================] - 0s - loss: 1.3073 - acc: 0.9987     
Epoch 179/300
2294/2294 [==============================] - 0s - loss: 1.3059 - acc: 0.9965     
Epoch 180/300
2294/2294 [==============================] - 0s - loss: 1.3013 - acc: 0.9983     
Epoch 181/300
2294/2294 [==============================] - 0s - loss: 1.2984 - acc: 0.9983     
Epoch 182/300
2294/2294 [=============================

2294/2294 [==============================] - 0s - loss: 1.1105 - acc: 0.9978     
Epoch 258/300
2294/2294 [==============================] - 0s - loss: 1.1071 - acc: 0.9983     
Epoch 259/300
2294/2294 [==============================] - 0s - loss: 1.1070 - acc: 0.9987     
Epoch 260/300
2294/2294 [==============================] - 0s - loss: 1.1070 - acc: 0.9974     
Epoch 261/300
2294/2294 [==============================] - 0s - loss: 1.1031 - acc: 0.9978     
Epoch 262/300
2294/2294 [==============================] - 0s - loss: 1.1011 - acc: 0.9974     
Epoch 263/300
2294/2294 [==============================] - 0s - loss: 1.0981 - acc: 0.9969     
Epoch 264/300
2294/2294 [==============================] - 0s - loss: 1.0978 - acc: 0.9983     
Epoch 265/300
2294/2294 [==============================] - 0s - loss: 1.0936 - acc: 0.9991     
Epoch 266/300
2294/2294 [==============================] - 0s - loss: 1.0960 - acc: 0.9961     
Epoch 267/300
2294/2294 [=============================

2294/2294 [==============================] - 0s - loss: 2.5148 - acc: 0.9878     
Epoch 41/300
2294/2294 [==============================] - 0s - loss: 2.4840 - acc: 0.9874     
Epoch 42/300
2294/2294 [==============================] - 0s - loss: 2.4583 - acc: 0.9856     
Epoch 43/300
2294/2294 [==============================] - 0s - loss: 2.4320 - acc: 0.9878     
Epoch 44/300
2294/2294 [==============================] - 0s - loss: 2.4081 - acc: 0.9874     
Epoch 45/300
2294/2294 [==============================] - 0s - loss: 2.3788 - acc: 0.9878     
Epoch 46/300
2294/2294 [==============================] - 0s - loss: 2.3520 - acc: 0.9908     
Epoch 47/300
2294/2294 [==============================] - 0s - loss: 2.3309 - acc: 0.9891     
Epoch 48/300
2294/2294 [==============================] - 0s - loss: 2.3034 - acc: 0.9904     
Epoch 49/300
2294/2294 [==============================] - 0s - loss: 2.2821 - acc: 0.9917     
Epoch 50/300
2294/2294 [==============================] - 0s - 

...

2294/2294 [==============================] - 0s - loss: 1.2056 - acc: 0.9978     
Epoch 223/300
2294/2294 [==============================] - 0s - loss: 1.2071 - acc: 0.9978     
Epoch 224/300
2294/2294 [==============================] - 0s - loss: 1.2022 - acc: 0.9974     
Epoch 225/300
2294/2294 [==============================] - 0s - loss: 1.2015 - acc: 0.9969     
Epoch 226/300
2294/2294 [==============================] - 0s - loss: 1.1957 - acc: 0.9983     
Epoch 227/300
2294/2294 [==============================] - 0s - loss: 1.1985 - acc: 0.9978     
Epoch 228/300
2294/2294 [==============================] - 0s - loss: 1.1974 - acc: 0.9956     
Epoch 229/300
2294/2294 [==============================] - 0s - loss: 1.1926 - acc: 0.9969     
Epoch 230/300
2294/2294 [==============================] - 0s - loss: 1.1899 - acc: 0.9974     
Epoch 231/300
2294/2294 [==============================] - 0s - loss: 1.1876 - acc: 0.9978     
Epoch 232/300
2294/2294 [=============================

2295/2295 [==============================] - 0s - loss: 7.5923 - acc: 0.9468     
Epoch 6/300
2295/2295 [==============================] - 0s - loss: 6.9204 - acc: 0.9503     
Epoch 7/300
2295/2295 [==============================] - 0s - loss: 6.3826 - acc: 0.9490     
Epoch 8/300
2295/2295 [==============================] - 0s - loss: 5.9448 - acc: 0.9603     
Epoch 9/300
2295/2295 [==============================] - 0s - loss: 5.5799 - acc: 0.9586     
Epoch 10/300
2295/2295 [==============================] - 0s - loss: 5.2602 - acc: 0.9651     
Epoch 11/300
2295/2295 [==============================] - 0s - loss: 5.0002 - acc: 0.9630     
Epoch 12/300
2295/2295 [==============================] - 0s - loss: 4.7747 - acc: 0.9647     
Epoch 13/300
2295/2295 [==============================] - 0s - loss: 4.5685 - acc: 0.9682     
Epoch 14/300
2295/2295 [==============================] - 0s - loss: 4.3828 - acc: 0.9695     
Epoch 15/300
2295/2295 [==============================] - 0s - loss

2295/2295 [==============================] - 0s - loss: 1.7329 - acc: 0.9913     
Epoch 92/300
2295/2295 [==============================] - 0s - loss: 1.7179 - acc: 0.9952     
Epoch 93/300
2295/2295 [==============================] - 0s - loss: 1.7147 - acc: 0.9943     
Epoch 94/300
2295/2295 [==============================] - 0s - loss: 1.7051 - acc: 0.9943     
Epoch 95/300
2295/2295 [==============================] - 0s - loss: 1.6971 - acc: 0.9952     
Epoch 96/300
2295/2295 [==============================] - 0s - loss: 1.6904 - acc: 0.9935     
Epoch 97/300
2295/2295 [==============================] - 0s - loss: 1.6777 - acc: 0.9961     
Epoch 98/300
2295/2295 [==============================] - 0s - loss: 1.6739 - acc: 0.9956     
Epoch 99/300
2295/2295 [==============================] - 0s - loss: 1.6636 - acc: 0.9943     
Epoch 100/300
2295/2295 [==============================] - 0s - loss: 1.6565 - acc: 0.9952     
Epoch 101/300
2295/2295 [==============================] - 0s 

2295/2295 [==============================] - 0s - loss: 1.2901 - acc: 0.9983     
Epoch 177/300
2295/2295 [==============================] - 0s - loss: 1.2874 - acc: 0.9969     
Epoch 178/300
2295/2295 [==============================] - 0s - loss: 1.2857 - acc: 0.9978     
Epoch 179/300
2295/2295 [==============================] - 0s - loss: 1.2849 - acc: 0.9956     
Epoch 180/300
2295/2295 [==============================] - 0s - loss: 1.2771 - acc: 0.9965     
Epoch 181/300
2295/2295 [==============================] - 0s - loss: 1.2764 - acc: 0.9952     
Epoch 182/300
2295/2295 [==============================] - 0s - loss: 1.2693 - acc: 0.9983     
Epoch 183/300
2295/2295 [==============================] - 0s - loss: 1.2713 - acc: 0.9965     
Epoch 184/300
2295/2295 [==============================] - 0s - loss: 1.2659 - acc: 0.9969     
Epoch 185/300
2295/2295 [==============================] - 0s - loss: 1.2621 - acc: 0.9974     
Epoch 186/300
2295/2295 [=============================

2295/2295 [==============================] - 0s - loss: 1.0869 - acc: 0.9969     
Epoch 262/300
2295/2295 [==============================] - 0s - loss: 1.0830 - acc: 0.9983     
Epoch 263/300
2295/2295 [==============================] - 0s - loss: 1.0802 - acc: 0.9974     
Epoch 264/300
2295/2295 [==============================] - 0s - loss: 1.0792 - acc: 0.9978     
Epoch 265/300
2295/2295 [==============================] - 0s - loss: 1.0789 - acc: 0.9965     
Epoch 266/300
2295/2295 [==============================] - 0s - loss: 1.0770 - acc: 0.9974     
Epoch 267/300
2295/2295 [==============================] - 0s - loss: 1.0748 - acc: 0.9969     
Epoch 268/300
2295/2295 [==============================] - 0s - loss: 1.0712 - acc: 0.9983     
Epoch 269/300
2295/2295 [==============================] - 0s - loss: 1.0727 - acc: 0.9969     
Epoch 270/300
2295/2295 [==============================] - 0s - loss: 1.0688 - acc: 0.9978     
Epoch 271/300
2295/2295 [=============================